# Predictor

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing import timeseries_dataset_from_array

import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import time
import IPython
import IPython.display

In [2]:
from preprocess.data_generator import WindowSplitter
from nn.deepar import create_model
from preprocess.data_generator import DatasetGenerator

In [3]:
from dl.predictor import Predictor

In [4]:
HISTORICAL_CSV_FILE = "../data/data.csv"
RAW_CSV_FILE = '../data/preprocessed_data.csv'
MODEL_CHECKPOINT_PATH = "../checkpoints/deepar"

### Predictor

In [5]:
historical_df = pd.read_csv(HISTORICAL_CSV_FILE)
display(historical_df[:3])
len(historical_df)

,t,x,y,z,tx1,tx2,tx3,tx4,tx5,tx6,...,tx16,tx17,tx18,tx19,tx20,tx21,tx22,tx23,tx24,tx25
0,1.667897e+12,750.02,598.52,94.50,-55.0,-62.0,-67.0,-63.0,-66.0,-69.0,...,-74.0,-72.0,-70.0,-69.0,-68.0,-67.0,-57.0,-53.0,-73.0,-61.0
1,1.667897e+12,759.46,579.46,94.11,-55.0,-63.0,-67.0,-63.0,-66.0,-68.0,...,-77.0,-72.0,-66.0,-71.0,-57.0,-68.0,-62.0,-56.0,-74.0,-62.0
2,1.667897e+12,770.00,559.05,94.94,-56.0,-62.0,-60.0,-62.0,-58.0,-48.0,...,-74.0,-71.0,-47.0,-70.0,-60.0,-65.0,-61.0,-51.0,-73.0,-62.0


16936

In [6]:
class Predictor:
    def __init__(self, historical_df, checkpoint=MODEL_CHECKPOINT_PATH, max_window_size=300):
        self.historical_df = historical_df.drop(columns=['t'])
        self.epsilon = 1e-10
        self.mean = self.historical_df.mean()
        self.std = self.historical_df.std() + self.epsilon
        self.label_size = 3
        self.input_size = len(self.historical_df.columns[self.label_size:])
        self.columns = self.historical_df.columns
        self.measurement = {}
        self.init_measurement()
        self.max_window_size = max_window_size
        # model
        self.checkpoint = checkpoint
        self.model = None
        self.load_model()

    def summary(self):
        print('COLUMNS:', self.columns)
        print('MEAN:', list(self.mean))
        print('STD: ', list(self.std))
        print('LABEL SIZE: ', self.label_size)
        print('INPUT SIZE: ', self.input_size)
        print('RAW MEASUREMENT:', self.measurement)
        print('MAX WINDOW SIZE:', self.max_window_size)
        
    def init_measurement(self):
        for column in self.columns[self.label_size:]:
            self.measurement[column] = []
        
    def add_measurement(self, tx, rssi):
        if tx in self.measurement.keys():
            self.measurement[tx].append(rssi)
            if len(self.measurement[tx]) > self.max_window_size:
                self.measurement[tx] = self.measurement[tx][1:]
                
    def gen_input_seq(self):
        df = pd.DataFrame.from_dict(self.measurement)
        # normalize
        df = (df - self.mean[self.label_size:]) / self.std[self.label_size:]
        return tf.convert_to_tensor([df.to_numpy()])
    
    def load_model(self):
        self.model = create_model(dim_x=self.input_size, dim_z=self.label_size)
        # restore checkpoints if exist
        ckpt = tf.train.Checkpoint(model=self.model)
        ckpt_manager = tf.train.CheckpointManager(ckpt, self.checkpoint, max_to_keep=100)
        if ckpt_manager.latest_checkpoint:
            ckpt.restore(ckpt_manager.latest_checkpoint)
            print("Restoring checkpoint from {}".format(ckpt_manager.latest_checkpoint))

    def predict(self):
        inputs = self.gen_input_seq()
        mean, _ = self.model(inputs)
        return mean * self.std[:self.label_size] + self.mean[:self.label_size]
            
predictor = Predictor(historical_df=historical_df)
predictor.summary()

Restoring checkpoint from ../checkpoints/deepar\ckpt-89
COLUMNS: Index(['x', 'y', 'z', 'tx1', 'tx2', 'tx3', 'tx4', 'tx5', 'tx6', 'tx7', 'tx8',
       'tx9', 'tx10', 'tx11', 'tx12', 'tx13', 'tx14', 'tx15', 'tx16', 'tx17',
       'tx18', 'tx19', 'tx20', 'tx21', 'tx22', 'tx23', 'tx24', 'tx25'],
      dtype='object')
MEAN: [-39.31665151157298, -13.787049480396787, 98.34586974492206, -67.24563060935286, -181.3551606046292, -196.39354038734058, -199.06819792158714, -198.88137694851204, -197.6956778460085, -199.49391828058575, -196.68209730751062, -200.4466225791214, -199.78938356164383, -200.86637931034483, -198.23517949929146, -200.75986065186584, -200.548535663675, -199.28826169107228, -200.13804912612187, -200.81099433160134, -200.7686584789797, -200.68800188946622, -201.87346480869155, -203.29463863958432, -201.47537789324517, -202.4958667926311, -203.7778696268304, -202.91585970713274]
STD:  [525.4834934166739, 519.4598867822032, 2.805543258091719, 24.798604416990003, 92.41645728529215,

### Simulator

In [7]:
class World:
    def __init__(self, raw_df, predictor):
        self.raw_df = raw_df
        self.raw_df_size = len(self.raw_df)
        self.raw_df_iter = 0
        self.predictor = predictor
        self.label = []
        
    def get_measurement(self):
        if self.raw_df_iter > self.raw_df_size:
            return
        measurement = self.raw_df.iloc[[self.raw_df_iter]].values[0]
        self.raw_df_iter+=1
        tx = measurement[0]
        rssi = measurement[4]
        
        # add ground true
        last = None
        if len(self.label) > 0:
            last = self.label[-1]
        current = measurement[1:4]
        if (last != current).any():
            self.label.append(current)
            
        return tx.lower(), rssi
    
    def get_label(self):
        return np.array(self.label)
    
    def step(self):
        tx, rssi = self.get_measurement()
        predictor.add_measurement(tx, rssi)

In [8]:
raw_df = pd.read_csv(RAW_CSV_FILE)
# predictor = Predictor(on_predict=None, historical_csv_path=HISTORICAL_CSV_FILE, checkpoint=MODEL_CHECKPOINT_PATH, max_window_size=100)
predictor = Predictor(historical_df=historical_df)
world = World(raw_df=raw_df, predictor=predictor)
for i in range(9 * 500):
    world.step()

Restoring checkpoint from ../checkpoints/deepar\ckpt-89


In [9]:
for k, v in predictor.measurement.items():
    print(k, len(v))

tx1 300
tx2 201
tx3 185
tx4 171
tx5 187
tx6 166
tx7 175
tx8 178
tx9 174
tx10 168
tx11 170
tx12 195
tx13 161
tx14 167
tx15 173
tx16 179
tx17 166
tx18 174
tx19 170
tx20 172
tx21 141
tx22 146
tx23 157
tx24 142
tx25 131


In [10]:
result = predictor.predict()[0]
label = world.get_label()

ValueError: arrays must all be same length

In [ ]:
plt.figure(figsize=(20,10))
pred_len = len(result)
plt.scatter(result[:, 0].numpy(), result[:, 1].numpy(), s=10, marker='x', alpha=0.5)
plt.scatter(label[-pred_len:, 0], label[-pred_len:, 1], s=10, marker='o', alpha=0.5)
plt.scatter(result[-1, 0].numpy(), result[-1, 1].numpy(), s=50, marker='x', color='r', alpha=1)
plt.scatter(label[-1, 0], label[-1, 1], s=50, marker='o', color='g', alpha=1)
plt.xlabel('X Coordinate (cm)')
plt.ylabel('Y Coordinate (cm)')
plt.show()